In [ ]:
import os
import openai
import discord
import nest_asyncio
from discord.ext import commands
from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()
DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY
intents = discord.Intents.default()
intents.messages = True
intents.message_content = True  # Add this line


bot = commands.Bot(command_prefix="", intents=intents)

async def generate_response(prompt):
    model_engine = "text-davinci-003"
    
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5,
    )

    message = completions.choices[0].text.strip()
    return message


# Event handler for processing messages
@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    # Check for image and video attachments
    image_url = None
    video_url = None
    if message.attachments:
        for attachment in message.attachments:
            if attachment.url.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_url = attachment.url
            elif attachment.url.lower().endswith(('.mp4', '.webm', '.mov', '.avi', '.mkv')):
                video_url = attachment.url

            if image_url and video_url:
                break

    # Create prompt for the model based on attachments
    prompt = message.content
    if image_url:
        prompt = f"An image with URL {image_url} and text: {prompt}"
    if video_url:
        prompt = f"A video with URL {video_url} and text: {prompt}"

    # Generate response and send it to the channel
    response = await generate_response(prompt)
    await message.channel.send(response)

# Run the bot
bot.run(DISCORD_TOKEN)
